In [6]:
import pandas as pd
import seaborn as sns 
from pandas import DataFrame
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
import numpy as np
import plotly
import statistics
import plotly.express as px
import stats
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import plotly.express as px
import datetime
import datetime as dt
import time


# Supress Scientific notation in python
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Display all columns of long dataframe
pd.set_option('display.max_columns', None)

#data from Kaggle.com
#open file
stocks='stocks.csv'
df=pd.read_csv(stocks)
#print(df)

In [12]:
#parse index
df['Date']=pd.to_datetime(df['Date'], infer_datetime_format=True)
indexeddf=df.set_index(['Date'])
#print(indexeddf)

#parsing to time format and extracting dates 
x=df['Date']=pd.to_datetime(df['Date'], format='%d-%m-%y')

Day=df['Date'].dt.day_name()
Month=df['Date'].dt.month_name()
Year=df['Date'].dt.year

df['Year']=df['Date'].dt.year
df['Month']=df['Date'].dt.month_name()
df['Day']=df['Date'].dt.day_name()
#print(df)

In [13]:
def get_month(x):
    return dt.datetime (x.year, x.month, 1)

df['StockMonth']=df['Date'].apply(get_month)
grouping=df.groupby('Code')['StockMonth']
df['CohortMonth']=grouping.transform('min')

# Calculate offset

In [16]:
def get_month_int(cohortframe, column):
    """function to generate date columns"""
    year=cohortframe[column].dt.year
    month=cohortframe[column].dt.month
    day=cohortframe[column].dt.day
    return year, month, day

    
#call function 
stock_year, stock_month, stock_day_=get_month_int(df,'StockMonth')
cohort_year, cohort_month,cohort_day_=get_month_int(df,'CohortMonth')

#create year an month diffs
year_diff=stock_year-cohort_year
month_diff=stock_month-cohort_month

# Create cohort index

In [17]:
df['CohortIndex']=year_diff * 12 + month_diff +1 

# Monthly cohort count

In [37]:
grouping = df.groupby(['StockMonth', 'CohortIndex'])
cohort_data = grouping['Code'].apply(pd.Series.nunique)

#return number of unique vals
cohort_data = cohort_data.reset_index()
cohort_counts = cohort_data.pivot(index='StockMonth', columns='CohortIndex', values='Code')
print(cohort_counts)

CohortIndex   1     2     3     4     5    6    7    8    9    10   11   12   \
StockMonth                                                                     
2011-12-01  32.00   nan   nan   nan   nan  nan  nan  nan  nan  nan  nan  nan   
2012-01-01  27.00 32.00   nan   nan   nan  nan  nan  nan  nan  nan  nan  nan   
2012-02-01  10.00 26.00 32.00   nan   nan  nan  nan  nan  nan  nan  nan  nan   
2012-03-01  51.00 10.00 27.00 32.00   nan  nan  nan  nan  nan  nan  nan  nan   
2012-04-01  57.00 51.00 10.00 27.00 32.00  nan  nan  nan  nan  nan  nan  nan   
...           ...   ...   ...   ...   ...  ...  ...  ...  ...  ...  ...  ...   
2020-11-01    nan  1.00   nan  1.00   nan 1.00  nan  nan  nan  nan  nan 1.00   
2020-12-01    nan   nan  1.00   nan  1.00  nan 1.00  nan  nan  nan  nan  nan   
2021-01-01    nan   nan   nan  1.00   nan 1.00  nan 1.00  nan  nan  nan  nan   
2021-02-01    nan   nan   nan   nan  1.00  nan 1.00  nan 1.00  nan  nan  nan   
2021-03-01    nan   nan   nan   nan   na